  
The cell below imports three modules from the langchain package.  
**Ollama**: Allows us to run open source large language models (such as Llama 2) locally.  
**CallbackManager**: A class from Langchain that manages the callbacks used during the training or inference process. It allows you to define multiple callbacks and execute them in a specific order.  
**StreamingStdOutCallbackHandler**: A class from Langchain that defines a callback handler that prints the output to the standard output stream in real-time.  
**FewShotPromptTemplate and PromptTemplate**: Classes from LangChain used to define reusable templates for generating prompts for few-shot learning and predictions with large language models.  
**LLMChain**: Class from LangChain used to chain together multiple LLMs sequentially to process an input.

In [13]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

Initializing an Ollama LLM instance with the LLaMA-2 model and configuring it to use the StreamingStdOutCallbackHandler for streaming predictions.

In [14]:
llm = Ollama(model='llama2', callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

Defining the labelled dataset for training

In [15]:
examples = [
    {"phrase": "The mess food is delicious, and I look forward to every meal.", "class": "Alpha"},
    {"phrase": "I find the mess food to be quite bland and uninspiring.", "class": "Beta"},
    {"phrase": "The mess staff is attentive, and they cater to dietary preferences.", "class": "Alpha"},
    {"phrase": "I often skip meals at the mess because the food quality is inconsistent.", "class": "Beta"},
    {"phrase": "The variety of options in the mess menu is impressive.", "class": "Alpha"},
    {"phrase": "I'm disappointed with the lack of vegetarian options in the mess.", "class": "Beta"},
    {"phrase": "The mess provides a convenient and affordable dining option.", "class": "Alpha"},
    {"phrase": "The mess food is not worth the money; I'd rather eat elsewhere.", "class": "Beta"},
    {"phrase": "I appreciate the effort the mess puts into themed dinners.", "class": "Alpha"},
    {"phrase": "The mess cleanliness standards need improvement.", "class": "Beta"},
    {"phrase": "The mess desserts are a delightful way to end the meal.", "class": "Alpha"},
    {"phrase": "The mess food makes me feel sick; I avoid it whenever I can.", "class": "Beta"},
    {"phrase": "The mess accommodates special dietary needs well.", "class": "Alpha"},
    {"phrase": "The mess food is repetitive, and I get tired of eating the same things.", "class": "Beta"},
    {"phrase": "The mess atmosphere is lively and encourages socializing.", "class": "Alpha"},
    {"phrase": "The mess food lacks freshness; it tastes like leftovers.", "class": "Beta"},
    {"phrase": "The mess timings are convenient for my schedule.", "class": "Alpha"},
    {"phrase": "I've had multiple instances of finding foreign objects in the mess food.", "class": "Beta"},
    {"phrase": "The mess food caters to a diverse range of tastes.", "class": "Alpha"},
    {"phrase": "The mess portion sizes are too small; I'm often left hungry.", "class": "Beta"},
    {"phrase": "I enjoy the cultural diversity reflected in the mess cuisine.", "class": "Alpha"},
    {"phrase": "The mess food is overpriced for its quality.", "class": "Beta"},
    {"phrase": "The mess hygiene standards are commendable.", "class": "Alpha"},
    {"phrase": "The mess food lacks nutritional value.", "class": "Beta"},
    {"phrase": "The mess offers a good balance between healthy and indulgent options.", "class": "Alpha"},
    {"phrase": "I can't stand the taste of the mess food; it's unbearable.", "class": "Beta"},
    {"phrase": "The mess takes feedback seriously and makes improvements.", "class": "Alpha"},
    {"phrase": "The mess service is slow, and I don't have time to wait.", "class": "Beta"},
    {"phrase": "The mess caters to different dietary preferences, including jain options.", "class": "Alpha"},
    {"phrase": "The mess food is too oily; it affects my health negatively.", "class": "Beta"},
]

Defining a template called example_template that can be used to format labeled examples for few-shot learning in Jinja2 format.

In [16]:
example_template = """
{{phrase}}
Class: {{class}}
"""

Creating a PromptTemplate from the example_template defined previously, specifying that the template uses Jinja2 syntax.

In [17]:
prompt = PromptTemplate.from_template(example_template, template_format='jinja2')

Defining a FewShotPromptTemplate using the previous PromptTemplate to format the examples.

The following arguments are given:-  
**examples** - Provides the examples to use  
**example_prompt** - The PromptTemplate to format each example  
**prefix** - Text before the examples to describe the task  
**suffix** - Text after the examples to prompt for a prediction  
**input_variables** - Variables that will be filled later ({input})  
**example_separator** - Separator between each example  

In [18]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt,
    prefix="Few-shot classification, only provide one word answer, no explanation needed.",
    suffix="Find the class of this phrase: {input}, based on the previous examples.",
    input_variables=["input"],
    example_separator="\n\n",
)

This shows a sample of the prompt generated using the format defined above.

In [19]:
print(few_shot_prompt.format(input="I eat the mess food only because I don't have any other option"))

Few-shot classification, only provide one word answer, no explanation needed.


The mess food is delicious, and I look forward to every meal.
Class: Alpha


I find the mess food to be quite bland and uninspiring.
Class: Beta


The mess staff is attentive, and they cater to dietary preferences.
Class: Alpha


I often skip meals at the mess because the food quality is inconsistent.
Class: Beta


The variety of options in the mess menu is impressive.
Class: Alpha


I'm disappointed with the lack of vegetarian options in the mess.
Class: Beta


The mess provides a convenient and affordable dining option.
Class: Alpha


The mess food is not worth the money; I'd rather eat elsewhere.
Class: Beta


I appreciate the effort the mess puts into themed dinners.
Class: Alpha


The mess cleanliness standards need improvement.
Class: Beta


The mess desserts are a delightful way to end the meal.
Class: Alpha


The mess food makes me feel sick; I avoid it whenever I can.
Class: Beta


The mess accommo

Creating an LLMChain with a single LLM (llm) and sets the prompt to the few_shot_prompt we defined earlier.

In [20]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)

Passing the input sentence to the predict method of our pre-configured LLMChain. This chain encapsulated the few-shot learning prompt we had defined earlier using a FewShotPromptTemplate, including the formatted examples and task description.

In [29]:
chain.predict(input="I eat the mess food only because I don't have any other option.")

Class: Beta

'Class: Beta'

In [35]:
chain.predict(input="The mess consistently impresses with delicious food, attentive staff, and a welcoming atmosphere.")

Alpha

'Alpha'

In [23]:
chain.predict(input="The mess food, while sometimes lacking in variety, manages to surprise my taste buds with unexpected flavors")

Alpha

'Alpha'

In [33]:
chain.predict(input="I appreciate the mess staff's dedication to cleanliness and hygiene standards, but the taste of the food is hit-or-miss.")

Alpha

'Alpha'

In [34]:
chain.predict(input="I appreciate the mess staff's dedication to cleanliness and hygiene standards, but the taste of the food is hit-or-miss.")

Class: Beta

'Class: Beta'

In [25]:
chain.predict(input="The mess desserts are usually a delightful way to end the meal, but occasionally, there's a letdown in terms of freshness.")

Alpha

'Alpha'

In [30]:
chain.predict(input="The mess food is kind of dull, not much to rave about. It's like a flavor snooze-fest.")

Class: Beta

'Class: Beta'

In [31]:
chain.predict(input="The mess food is like a daily surprise party for my taste buds—unpredictable and not always in a good way. Bon appétit, or should I say, bon chance!")

Alpha

'Alpha'

In [32]:
chain.predict(input="The mess food is a real adventure in mediocrity. It's like they have a secret mission to make every dish taste just okay. ")

Class: Beta

'Class: Beta'

The evaluation of the llama2 model on the provided examples indicates a robust understanding of classifying phrases into distinct categories. However, challenges emerge in instances where there's a mixture of positive and negative sentiments within a single statement. For example, in the phrase "I appreciate the mess staff's dedication to cleanliness and hygiene standards, but the taste of the food is hit-or-miss." the model exhibits variability in its predictions across repeated trials.    
Moreover, the model faces difficulty in accurately classifying sarcastic expressions, as evident in the statement: "The mess food is like a daily surprise party for my taste buds—unpredictable and not always in a good way. Bon appétit, or should I say, bon chance!" Sarcasm, with its nuanced and ironic tone, appears to pose a challenge for the model's classification accuracy.    
These observations suggest that while the llama2 model demonstrates a commendable grasp of classifying sentiment, improvements may be needed to handle cases of mixed sentiments and nuances in sarcastic expressions.